<a href="https://colab.research.google.com/github/shrutishrinivasan/CIA_DAA/blob/main/ParticleSwarmOptimization(NN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
data = pd.read_csv("loan.csv")
data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [ ]:
df = data
# removing irrelevant columns like Id, Zip code
# and shifting the target variable to the last position
df = df.drop(['ID','ZIP Code','Personal Loan'],axis=1)
# Target variable: Personal Loan
df['Perosnal Loan'] = data['Personal Loan']
df.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard,Perosnal Loan
0,25,1,49,4,1.6,1,0,1,0,0,0,0
1,45,19,34,3,1.5,1,0,1,0,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,1,0


In [ ]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0,test_size=0.25)
X_train

array([[ 51.,  25., 163., ...,   0.,   1.,   0.],
       [ 42.,  17.,  44., ...,   1.,   1.,   1.],
       [ 35.,  11.,  24., ...,   0.,   0.,   0.],
       ...,
       [ 26.,   1.,  24., ...,   0.,   0.,   1.],
       [ 57.,  33.,  49., ...,   1.,   1.,   1.],
       [ 33.,   9.,  38., ...,   0.,   0.,   0.]])

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# Define the number of particles and iterations
num_particles = 50
num_iterations = 100

In [ ]:
import tensorflow as tf

In [ ]:
# Define the neural network architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(10,)),
    tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

# Define the fitness function
def fitness_function(params):
    # Set the model weights to the particle's parameters
    model.set_weights(params)
    
    # Train the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, verbose=0)
    
    # Evaluate the model on the testing set
    y_pred = np.argmax(model.predict(X_test), axis=-1)
    return -accuracy_score(y_test, y_pred)  # negative accuracy to minimize


In [ ]:
# Initialize the particles' positions and velocities
positions = [model.get_weights() for _ in range(num_particles)]
velocities = [np.random.uniform(-1, 1, p.shape) for p in positions]
pbest_positions = positions.copy()
pbest_fitness = [fitness_function(p) for p in pbest_positions]
gbest_position = pbest_positions[np.argmin(pbest_fitness)]
gbest_fitness = np.min(pbest_fitness)

In [ ]:
# Perform PSO iterations
for i in range(num_iterations):
    for j in range(num_particles):
        # Update the particle's velocity and position
        velocities[j] = 0.5 * velocities[j] + 1.5 * np.random.uniform(0, 1, positions[j].shape) * (pbest_positions[j] - positions[j]) + 1.5 * np.random.uniform(0, 1, positions[j].shape) * (gbest_position - positions[j])
        positions[j] = positions[j] + velocities[j]
        
        # Evaluate the fitness of the new position
        fitness = fitness_function(positions[j])
        
        # Update the particle's personal best and global best
        if fitness < pbest_fitness[j]:
            pbest_positions[j] = positions[j]
            pbest_fitness[j] = fitness
            if fitness < gbest_fitness:
                gbest_position = positions[j]
                gbest_fitness = fitness

In [22]:
# Set the model weights to the global best position
model.set_weights(gbest_position)

# Evaluate the model on the testing set
y_pred = np.argmax(model.predict(X_test), axis=-1)
print(f"Accuracy = {accuracy_score(y_test, y_pred):.4f}")

Accuracy = 0.9208
